In [4]:
import requests
import pandas as pd
from parsel import Selector
import re
import time

# 抓取商品链接

In [5]:
# 数据存储字典
d = {"商品信息": [],
     "西域订货号": [],
     "链接": []
    }

# 设置请求头模拟浏览器访问
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Referer': 'https://www.ehsy.com/',
    'Accept-Language': 'zh-CN,zh;q=0.9'
}

for p in range(1, 9):
    url = f"https://www.ehsy.com/search?k=%E8%9E%BA%E4%B8%9D%E9%92%89&p={str(p)}" # 不同商品的k 不同
    response = requests.get(url, headers=headers, timeout=10)
    # 创建选择器
    sel = Selector(response.text)
    for div in sel.css("div.product"):
        d["商品信息"].append(div.css("a::attr('title')").get().strip())
        d["西域订货号"].append(div.css("::attr('data-text')").get().strip())
        d["链接"].append(div.css("a::attr('href')").get().strip())
#         li_sel_list = div.css("ul.product-parameter>li")
#         for key, li_sel in zip(index, li_sel_list):
#             if key == "价格":
#                 key = "西域价"
#             value_list = li_sel.css("::text").getall()
#             value = " ".join([re.sub(r"\s+", "", str(i)) for i in value_list])
#             value = value.strip()
#             if len(value) == 0:
#                 value = None
#             d[key].append(value)
#             for other_key in list(d.keys())[3:]:
#                 if (other_key not in index) & (other_key != "西域价"):
#                     d[other_key].append(None)

In [7]:
# 存储链接数据
df = pd.DataFrame(d)
df.to_excel(r"../data/西域商品螺丝钉链接数据.xlsx")

In [2]:
df = pd.read_excel(r"../data/西域商品电池链接数据.xlsx", index_col=0)
df

,商品信息,西域订货号,链接
0,超霸 方形碳性电池，1604G 6F22 9V 售卖规格：1粒,JQH490,https://www.ehsy.com/product-JQH490
1,松下/Panasonic 大号（1号）碳性电池，R20NU/2S 售卖规格：2节/卡,EDG759,https://www.ehsy.com/product-EDG759
2,超霸 1号碳性电池，13G-BJ2 售卖规格：2节/卡,EDH182,https://www.ehsy.com/product-EDH182
3,超霸 2号碳性电池，R14P 2号 售卖规格：2粒/包,BEC883,https://www.ehsy.com/product-BEC883
4,南孚 镍氢充电电池，2050mAh 5# 5号充电电池KTV无线麦克风专用（20节/盒） 售...,HJP615,https://www.ehsy.com/product-HJP615
...,...,...,...
3595,明纬 开关电源，HDR-15-24 售卖规格：1个,ART872,https://www.ehsy.com/product-ART872
3596,明纬 开关电源，HDR-30-12 售卖规格：1个,ART875,https://www.ehsy.com/product-ART875
3597,明纬 开关电源，HDR-30-48 售卖规格：1个,ART878,https://www.ehsy.com/product-ART878
3598,明纬 开关电源，HDR-30-15 售卖规格：1个,ART876,https://www.ehsy.com/product-ART876


# 根据链接数据抓取商品属性

In [9]:
import asyncio
import httpx
import pandas as pd
from tqdm.notebook import tqdm
import random

headers1 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Referer': 'https://www.ehsy.com/',
    'Accept-Language': 'zh-CN,zh;q=0.9'
}
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
     'Referer': 'https://www.ehsy.com/',
    'Accept-Language': 'zh-CN,zh;q=0.9'
}
headers3 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.2151.72',
    'Referer': 'https://www.ehsy.com/',
    'Accept-Language': 'zh-CN,zh;q=0.9'
}
headers = [headers1, headers2, headers3]
async def fetch_data(client, semaphore, item, pbar):
    async with semaphore:
        try:
            response = await client.get(item["链接"], headers=random.choice(headers), timeout=5)
            time.sleep(random.choice([0, 1, 2]))
            sel = Selector(text=response.text)
            tr_list = sel.css(".standardTable tr.keyValue")
            for td in tr_list:
                key, value = td.css("td::text").extract()
                item[key.strip()] = value.strip()
            pbar.update(1)
            return item
        except Exception as e:
            print(f"请求失败: {item['链接']}，错误信息: {str(e)}")
            return None

async def main():
    df = pd.read_excel(r"../data/西域商品螺丝钉链接数据.xlsx", index_col=0)
    data_list = []
    semaphore = asyncio.Semaphore(10)
    with tqdm(total=len(df), desc="抓取进度") as pbar:
        async with httpx.AsyncClient() as client:
            tasks = []
            for i in df.itertuples(index=True):
                item = {
                    "商品信息": i.商品信息,
                    "西域订货号": i.西域订货号,
                    "链接": i.链接
                }
                task = fetch_data(client, semaphore, item, pbar)
                tasks.append(task)
            results = await asyncio.gather(*tasks)
    data_list = [result for result in results if result]
    df = pd.DataFrame(data_list)
    df.to_excel(r"../data/西域商品螺丝钉详细数据.xlsx")
    return df

if __name__ == "__main__":
    # 普通脚本环境
#     asyncio.run(main())
    result = await main()

# 如果在 Jupyter 等交互式环境中运行，请改用：
# await main()

抓取进度:   0%|          | 0/264 [00:00<?, ?it/s]